## Homework: Multilingual Embedding-based Machine Translation (7 points)

**In this homework** **<font color='red'>YOU</font>** will make machine translation system without using parallel corpora, alignment, attention, 100500 depth super-cool recurrent neural network and all that kind superstuff.

But even without parallel corpora this system can be good enough (hopefully). 

For our system we choose two kindred Slavic languages: Ukrainian and Russian. 

### Feel the difference!

(_синій кіт_ vs. _синій кит_)

![blue_cat_blue_whale.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/blue_cat_blue_whale.png)

### Frament of the Swadesh list for some slavic languages

The Swadesh list is a lexicostatistical stuff. It's named after American linguist Morris Swadesh and contains basic lexis. This list are used to define subgroupings of languages, its relatedness.

So we can see some kind of word invariance for different Slavic languages.


| Russian         | Belorussian              | Ukrainian               | Polish             | Czech                         | Bulgarian            |
|-----------------|--------------------------|-------------------------|--------------------|-------------------------------|-----------------------|
| женщина         | жанчына, кабета, баба    | жінка                   | kobieta            | žena                          | жена                  |
| мужчина         | мужчына                  | чоловік, мужчина        | mężczyzna          | muž                           | мъж                   |
| человек         | чалавек                  | людина, чоловік         | człowiek           | člověk                        | човек                 |
| ребёнок, дитя   | дзіця, дзіцёнак, немаўля | дитина, дитя            | dziecko            | dítě                          | дете                  |
| жена            | жонка                    | дружина, жінка          | żona               | žena, manželka, choť          | съпруга, жена         |
| муж             | муж, гаспадар            | чоловiк, муж            | mąż                | muž, manžel, choť             | съпруг, мъж           |
| мать, мама      | маці, матка              | мати, матір, неня, мама | matka              | matka, máma, 'стар.' mateř    | майка                 |
| отец, тятя      | бацька, тата             | батько, тато, татусь    | ojciec             | otec                          | баща, татко           |
| много           | шмат, багата             | багато                  | wiele              | mnoho, hodně                  | много                 |
| несколько       | некалькі, колькі         | декілька, кілька        | kilka              | několik, pár, trocha          | няколко               |
| другой, иной    | іншы                     | інший                   | inny               | druhý, jiný                   | друг                  |
| зверь, животное | жывёла, звер, істота     | тварина, звір           | zwierzę            | zvíře                         | животно               |
| рыба            | рыба                     | риба                    | ryba               | ryba                          | риба                  |
| птица           | птушка                   | птах, птиця             | ptak               | pták                          | птица                 |
| собака, пёс     | сабака                   | собака, пес             | pies               | pes                           | куче, пес             |
| вошь            | вош                      | воша                    | wesz               | veš                           | въшка                 |
| змея, гад       | змяя                     | змія, гад               | wąż                | had                           | змия                  |
| червь, червяк   | чарвяк                   | хробак, черв'як         | robak              | červ                          | червей                |
| дерево          | дрэва                    | дерево                  | drzewo             | strom, dřevo                  | дърво                 |
| лес             | лес                      | ліс                     | las                | les                           | гора, лес             |
| палка           | кій, палка               | палиця                  | patyk, pręt, pałka | hůl, klacek, prut, kůl, pálka | палка, пръчка, бастун |

But the context distribution of these languages demonstrates even more invariance. And we can use this fact for our for our purposes.

## Data

In [4]:
# !pip3 install python-Levenshtein

In [5]:
import gensim
import numpy as np
from gensim.models import KeyedVectors

Download embeddings here:
* [cc.uk.300.vec.zip](https://yadi.sk/d/9CAeNsJiInoyUA)
* [cc.ru.300.vec.zip](https://yadi.sk/d/3yG0-M4M8fypeQ)

Load embeddings for ukrainian and russian.

In [6]:
uk_emb = KeyedVectors.load_word2vec_format("cc.uk.300.vec")

In [7]:
ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec")

In [8]:
ru_emb.most_similar([ru_emb["август"]], topn=10)

[('август', 1.0000001192092896),
 ('июль', 0.9383152723312378),
 ('сентябрь', 0.9240029454231262),
 ('июнь', 0.9222574830055237),
 ('октябрь', 0.9095539450645447),
 ('ноябрь', 0.8930036425590515),
 ('апрель', 0.8729087114334106),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545795679092407),
 ('февраль', 0.8401415944099426)]

In [9]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 0.9999998807907104),
 ('липень', 0.9096441268920898),
 ('вересень', 0.9016969203948975),
 ('червень', 0.8992518782615662),
 ('жовтень', 0.8810408115386963),
 ('листопад', 0.8787633180618286),
 ('квітень', 0.8592804670333862),
 ('грудень', 0.8586863279342651),
 ('травень', 0.840811014175415),
 ('лютий', 0.8256431221961975)]

In [10]:
ru_emb.most_similar([uk_emb["серпень"]])

[('Недопустимость', 0.24435284733772278),
 ('конструктивность', 0.23293082416057587),
 ('офор', 0.23256804049015045),
 ('deteydlya', 0.230317160487175),
 ('пресечении', 0.22632381319999695),
 ('одностороннего', 0.22608886659145355),
 ('подход', 0.2230587750673294),
 ('иболее', 0.22003726661205292),
 ('2015Александр', 0.21872766315937042),
 ('конструктивен', 0.21796567738056183)]

Load small dictionaries for correspoinding words pairs as trainset and testset.

In [11]:
def load_word_pairs_word2vec(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [12]:
uk_ru_train_word2vec, X_train_word2vec, Y_train_word2vec = load_word_pairs_word2vec("ukr_rus.train.txt")

In [13]:
uk_ru_test_word2vec, X_test_word2vec, Y_test_word2vec = load_word_pairs_word2vec("ukr_rus.test.txt")

## Embedding space mapping

Let $x_i \in \mathrm{R}^d$ be the distributed representation of word $i$ in the source language, and $y_i \in \mathrm{R}^d$ is the vector representation of its translation. Our purpose is to learn such linear transform $W$ that minimizes euclidian distance between $Wx_i$ and $y_i$ for some subset of word embeddings. Thus we can formulate so-called Procrustes problem:

$$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$$
or
$$W^*= \arg\min_W ||WX - Y||_F$$

where $||*||_F$ - Frobenius norm.

In Greek mythology, Procrustes or "the stretcher" was a rogue smith and bandit from Attica who attacked people by stretching them or cutting off their legs, so as to force them to fit the size of an iron bed. We make same bad things with source embedding space. Our Procrustean bed is target embedding space.

![embedding_mapping.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/embedding_mapping.png)

![procrustes.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/procrustes.png)

But wait...$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$ looks like simple multiple linear regression (without intercept fit). So let's code.

In [14]:
from sklearn.linear_model import LinearRegression

mapping_word2vec = LinearRegression().fit(X_train_word2vec, Y_train_word2vec)

Let's take a look at neigbours of the vector of word _"серпень"_ (_"август"_ in Russian) after linear transform.

In [15]:
august_word2vec = mapping_word2vec.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august_word2vec)

[('апрель', 0.8541592955589294),
 ('июнь', 0.8411963582038879),
 ('март', 0.8397400379180908),
 ('сентябрь', 0.8359215259552002),
 ('февраль', 0.8328747749328613),
 ('октябрь', 0.8311805725097656),
 ('ноябрь', 0.8278147578239441),
 ('июль', 0.823634922504425),
 ('август', 0.8120612502098083),
 ('декабрь', 0.8038000464439392)]

We can see that neighbourhood of this embedding cosists of different months, but right variant is on the ninth place.

As quality measure we will use precision top-1, top-5 and top-10 (for each transformed Ukrainian embedding we count how many right target pairs are found in top N nearest neighbours in Russian embedding space).

In [16]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        res = [value for (value, _) in ru_emb.most_similar([mapped_vectors[i]])]
        num_matches += 1 if ru in res[:topn] else 0
    precision_val = num_matches / len(pairs)
    return precision_val


In [17]:
assert precision([("серпень", "август")], august_word2vec, topn=5) == 0.0
assert precision([("серпень", "август")], august_word2vec, topn=9) == 1.0
assert precision([("серпень", "август")], august_word2vec, topn=10) == 1.0

In [18]:
assert precision(uk_ru_test_word2vec, X_test_word2vec) == 0.0
assert precision(uk_ru_test_word2vec, Y_test_word2vec) == 1.0

In [20]:
precision_top1 = precision(uk_ru_test_word2vec, mapping_word2vec.predict(X_test_word2vec), 1)
precision_top5 = precision(uk_ru_test_word2vec, mapping_word2vec.predict(X_test_word2vec), 5)

assert precision_top1 >= 0.635
assert precision_top5 >= 0.811

## Making it better (orthogonal Procrustean problem)

It can be shown (see original paper) that a self-consistent linear mapping between semantic spaces should be orthogonal. 
We can restrict transform $W$ to be orthogonal. Then we will solve next problem:

$$W^*= \arg\min_W ||WX - Y||_F \text{, where: } W^TW = I$$

$$I \text{- identity matrix}$$

Instead of making yet another regression problem we can find optimal orthogonal transformation using singular value decomposition. It turns out that optimal transformation $W^*$ can be expressed via SVD components:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$
$$W^*=UV^T$$

In [21]:
from sklearn.decomposition import TruncatedSVD

def learn_transform(X_train, Y_train):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    x_t_y = np.dot(np.transpose(X_train), Y_train)
    u, s, vh = np.linalg.svd(x_t_y, full_matrices=True)
    w = np.dot(u, vh)
    return w

In [22]:
W_word2vec = learn_transform(X_train_word2vec, Y_train_word2vec)

In [24]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W_word2vec)])

[('апрель', 0.8237906694412231),
 ('сентябрь', 0.8049713373184204),
 ('март', 0.8025653958320618),
 ('июнь', 0.8021842837333679),
 ('октябрь', 0.8001735806465149),
 ('ноябрь', 0.7934484481811523),
 ('февраль', 0.7914121150970459),
 ('июль', 0.7908110022544861),
 ('август', 0.7891016602516174),
 ('декабрь', 0.7686373591423035)]

In [25]:
assert precision(uk_ru_test_word2vec, np.matmul(X_test_word2vec, W_word2vec)) >= 0.653
assert precision(uk_ru_test_word2vec, np.matmul(X_test_word2vec, W_word2vec), 5) >= 0.824

## UK-RU Translator

Now we are ready to make simple word-based translator: for earch word in source language in shared embedding space we find the nearest in target language.


In [26]:
with open("fairy_tale.txt", "r") as inpf:
    uk_sentences = [line.rstrip().lower() for line in inpf]

In [27]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()
    
def translate_word2vec(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    ru_sentence = ""

    vector = np.zeros([uk_emb.vector_size], dtype='float32')
    word_vectors = []

    data_tok = tokenizer.tokenize(sentence.lower())
    word_vectors = np.array([uk_emb.get_vector(word) for word in data_tok if word in uk_emb.key_to_index.keys()])
    ru_sentence = " ".join([ru_emb.most_similar([np.matmul(vector, W_word2vec)])[0][0] for vector in word_vectors])
    
    return ru_sentence

In [28]:
assert translate_word2vec(".") == "."
assert translate_word2vec("1 , 3") == "1 , 3"
assert translate_word2vec("кіт зловив мишу") == "кот поймал мышку"

In [29]:
for sentence in uk_sentences:
    print("src: {}\ndst: {}\n".format(sentence, translate_word2vec(sentence)))

src: лисичка - сестричка і вовк - панібрат
dst: лисичка – сестричка и волк –

src: як була собі лисичка та зробила хатку, та й живе. а це приходять холоди. от лисичка замерзла та й побігла в село вогню добувать, щоб витопити. прибігає до одної баби та й каже:
dst: как была себе лисичка и сделала избушку , и и живет . а оно приходят морозы . из лисичка замерзла и и побежала во село огня , чтобы . прибегает к одной бабы и и говорит :

src: — здорові були, бабусю! з неділею... позичте мені огню, я вам одслужу.
dst: — здоровые были , бабушку ! со воскресеньем ... мне огня , мной тебе .

src: — добре, — каже, — лисичко - сестричко. сідай погрійся трохи, поки я пиріжечки повибираю з печі!
dst: — хорошо , — говорит , — – сестричка . садись немного , пока мной пирожки со печи !

src: а баба макові пиріжки пекла. от баба вибирає пиріжки та на столі кладе, щоб прохололи; а лисичка підгляділа та за пиріг, та з хати... виїла мачок із середини, а туди напхала сміттячка, стулила та й біжить.
dst: а 

Not so bad, right? We can easily improve translation using language model and not one but several nearest neighbours in shared embedding space. But next time.

## Additional investigation

Using fastText implementation from Gensim due to better support for similarity methods

In [30]:
from gensim.models.fasttext import load_facebook_model

### Loading models

In [31]:
# downloading models
# fasttext.util.download_model('uk', if_exists='ignore')  # English
# fasttext.util.download_model('ru', if_exists='ignore')  # English

In [32]:
uk_ft = load_facebook_model('cc.uk.300.bin')

In [33]:
ru_ft = load_facebook_model('cc.ru.300.bin')

In [34]:
print("Number of words in ukranian fastText model", len(uk_ft.wv))
print("Number of words in russian fastText model", len(ru_ft.wv))

Number of words in ukranian fastText model 2000000
Number of words in russian fastText model 2000000


In [35]:
print("Number of words in ukranian word2vec model", len(uk_emb.index_to_key))
print("Number of words in russian word2vec model", len(ru_emb.index_to_key))

Number of words in ukranian word2vec model 600000
Number of words in russian word2vec model 600000


We can see that in fastText model there are almost 3 times more words than in word2vec model. So the quality of traslation can be even better. Let's find out!

In [36]:
def load_word_pairs_fasttext(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_ft.wv[uk])
            ru_vectors.append(ru_ft.wv[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [37]:
uk_ru_train_fasttext, X_train_fasttext, Y_train_fasttext = load_word_pairs_fasttext("ukr_rus.train.txt")

In [38]:
uk_ru_test_fasttext, X_test_fasttext, Y_test_fasttext = load_word_pairs_fasttext("ukr_rus.test.txt")

In [39]:
W_fasttext = learn_transform(X_train_fasttext, Y_train_fasttext)

In [40]:
def translate_fasttext(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    ru_sentence = ""

    vector = np.zeros([uk_emb.vector_size], dtype='float32')
    word_vectors = []

    data_tok = tokenizer.tokenize(sentence.lower())
    word_vectors = np.array([uk_ft.wv[word] for word in data_tok if word in uk_ft.wv])
    ru_sentence = " ".join([ru_ft.wv.most_similar([np.matmul(vector, W_word2vec)])[0][0] for vector in word_vectors])
    
    return ru_sentence

In [41]:
for sentence in uk_sentences:
    print("src: {}\ndst: {}\n".format(sentence, translate_fasttext(sentence)))

src: лисичка - сестричка і вовк - панібрат
dst: лисичка – сестричка и волк – сын

src: як була собі лисичка та зробила хатку, та й живе. а це приходять холоди. от лисичка замерзла та й побігла в село вогню добувать, щоб витопити. прибігає до одної баби та й каже:
dst: как была себе лисичка и сделала хатку , и и живет . а оно приходят морозы . из лисичка замерзла и и побежала во село огня добывать , чтобы истопить . прибегает к одной бабы и и говорит :

src: — здорові були, бабусю! з неділею... позичте мені огню, я вам одслужу.
dst: — здоровые были , бабушку ! со Пасхою ... одолжу мне огня , мной тебе присягаю .

src: — добре, — каже, — лисичко - сестричко. сідай погрійся трохи, поки я пиріжечки повибираю з печі!
dst: — хорошо , — говорит , — лисонька – сестричка . садись покупатьCloseвосстановление немного , пока мной пирожки выбираю со печи !

src: а баба макові пиріжки пекла. от баба вибирає пиріжки та на столі кладе, щоб прохололи; а лисичка підгляділа та за пиріг, та з хати... виїл

## Would you like to learn more?

### Articles:
* [Exploiting Similarities among Languages for Machine Translation](https://arxiv.org/pdf/1309.4168)  - entry point for multilingual embedding studies by Tomas Mikolov (the author of W2V)
* [Offline bilingual word vectors, orthogonal transformations and the inverted softmax](https://arxiv.org/pdf/1702.03859) - orthogonal transform for unsupervised MT
* [Word Translation Without Parallel Data](https://arxiv.org/pdf/1710.04087)
* [Loss in Translation: Learning Bilingual Word Mapping with a Retrieval Criterion](https://arxiv.org/pdf/1804.07745)
* [Unsupervised Alignment of Embeddings with Wasserstein Procrustes](https://arxiv.org/pdf/1805.11222)

### Repos (with ready-to-use multilingual embeddings):
* https://github.com/facebookresearch/MUSE

* https://github.com/Babylonpartners/fastText_multilingual -